## Backtracking
What: Systematic exploration of solution spaces by incrementally building candidates and abandoning ("backtracking") those that cannot lead to valid solutions.

When to use:
- Combinatorial problems: generating all permutations, combinations, subsets
- Constraint satisfaction: N-Queens, Sudoku solving, graph coloring
- Path finding: finding all paths in a maze or graph with certain properties

Key insight: Build solutions incrementally; as soon as you determine a partial candidate cannot lead to a valid solution, abandon it and backtrack.

Template:
```python
def backtrack(path, choices):
    if is_complete(path):
        process_solution(path)
        return
    for choice in get_choices(path):
        if is_valid(path + [choice]):
            path.append(choice)
            backtrack(path, remaining_choices)
            path.pop()  # backtrack
```

Complexity: Exponential in worst case, but pruning can dramatically reduce the search space.

Edge cases: Empty input, constraints that make no solution possible, duplicate handling.

In [2]:
use std::collections::HashSet;

// 1) Subsets (power set)
fn subsets(nums: &[i32]) -> Vec<Vec<i32>> {
    let mut res: Vec<Vec<i32>> = Vec::new();
    let mut path: Vec<i32> = Vec::new();

    fn dfs(nums: &[i32], path: &mut Vec<i32>, res: &mut Vec<Vec<i32>>, i: usize) {
        if i == nums.len() {
            res.push(path.clone());
            return;
        }
        // choice: include nums[i]
        path.push(nums[i]);
        dfs(nums, path, res, i + 1);
        path.pop();
        // choice: exclude nums[i]
        dfs(nums, path, res, i + 1);
    }

    dfs(nums, &mut path, &mut res, 0);
    res
}

println!("Subsets of [1,2,3]: {:?}", subsets(&[1, 2, 3]));


Subsets of [1,2,3]: [[1, 2, 3], [1, 2], [1, 3], [1], [2, 3], [2], [3], []]


In [5]:
// 2) Permutations
fn permute(nums: &[i32]) -> Vec<Vec<i32>> {
    let mut res: Vec<Vec<i32>> = Vec::new();
    let mut path: Vec<i32> = Vec::new();
    let mut used = vec![false; nums.len()];

    fn dfs(nums: &[i32], path: &mut Vec<i32>, used: &mut [bool], res: &mut Vec<Vec<i32>>) {
        if path.len() == nums.len() {
            res.push(path.clone());
            return;
        }
        for (i, &val) in nums.iter().enumerate() {
            if used[i] {
                continue;
            }
            used[i] = true;
            path.push(val);
            dfs(nums, path, used, res);
            path.pop();
            used[i] = false;
        }
    }

    dfs(nums, &mut path, &mut used, &mut res);
    res
}

let perms = permute(&[1, 2, 3]);
println!("Permutations of [1,2,3] (first 3): {:?}", &perms[..3.min(perms.len())]);

Permutations of [1,2,3] (first 3): [[1, 2, 3], [1, 3, 2], [2, 1, 3]]


In [9]:
// 3) N-Queens
fn solve_n_queens(n: usize) -> Vec<Vec<String>> {
    let mut res: Vec<Vec<String>> = Vec::new();
    let mut cols: HashSet<usize> = HashSet::new();
    let mut diag1: HashSet<i32> = HashSet::new(); // r - c
    let mut diag2: HashSet<usize> = HashSet::new(); // r + c
    let mut board = vec![vec!['.'; n]; n];

    fn dfs(
        board: &mut Vec<Vec<char>>,
        res: &mut Vec<Vec<String>>,
        cols: &mut HashSet<usize>,
        diag1: &mut HashSet<i32>,
        diag2: &mut HashSet<usize>,
        r: usize,
        n: usize,
    ) {
        if r == n {
            res.push(board.iter().map(|row| row.iter().collect()).collect());
            return;
        }
        for c in 0..n {
            if cols.contains(&c) || diag1.contains(&(r as i32 - c as i32)) || diag2.contains(&(r + c)) {
                continue;
            }
            cols.insert(c);
            diag1.insert(r as i32 - c as i32);
            diag2.insert(r + c);
            board[r][c] = 'Q';
            dfs(board, res, cols, diag1, diag2, r + 1, n);
            board[r][c] = '.';
            cols.remove(&c);
            diag1.remove(&(r as i32 - c as i32));
            diag2.remove(&(r + c));
        }
    }

    dfs(&mut board, &mut res, &mut cols, &mut diag1, &mut diag2, 0, n);
    res
}


println!("N-Queens n=4 solutions:");
for sol in solve_n_queens(4) {
    for row in sol {
        println!("{}", row);
    }
}


N-Queens n=4 solutions:
.Q..
...Q
Q...
..Q.
..Q.
Q...
...Q
.Q..


()

In [8]:
// 4) Combination Sum (repetition allowed)
fn combination_sum(candidates: &[i32], target: i32) -> Vec<Vec<i32>> {
    let mut candidates = candidates.to_vec();
    candidates.sort();
    let mut res: Vec<Vec<i32>> = Vec::new();
    let mut path: Vec<i32> = Vec::new();

    fn dfs(candidates: &[i32], path: &mut Vec<i32>, res: &mut Vec<Vec<i32>>, start: usize, remain: i32) {
        if remain == 0 {
            res.push(path.clone());
            return;
        }
        for i in start..candidates.len() {
            let c = candidates[i];
            if c > remain {
                break;
            }
            path.push(c);
            dfs(candidates, path, res, i, remain - c); // i again allows reuse
            path.pop();
        }
    }

    dfs(&candidates, &mut path, &mut res, 0, target);
    res
}


println!("Combination Sum for [2,3,6,7], target=7: {:?}", combination_sum(&[2,3,6,7], 7));


Combination Sum for [2,3,6,7], target=7: [[2, 2, 3], [7]]
